In [39]:
import numpy as np

In [84]:
class DP:
  def __init__(self, rewards, sp, method="policy", steps=10, delta=0.5, theta = 0.1):
    self.r = rewards
    self.sp = sp
    self.states = sp.shape[0]
    self.actions = sp.shape[1]
    
    self.p_a = np.full(sp.shape, 1/self.actions) # pi(a|s)
    self.v = np.zeros(self.states)               # v_pi
    self.delta = delta                           # discount rate
    self.theta = theta                           # error to stop
    self.steps = steps                           # max steps

    method_map = {
        "policy": self.policy_iteration,
        "value": self.value_iteration
    }
    self.method = method_map.get(method, None)

  def run(self):
    return self.method()

  def value_iteration(self):
    error = self.theta
    i = 0
    while error >= self.theta and i < self.steps:
      for s in range(len(self.v)):
        v = self.v[s]
        self.v[s] = np.max([self.r[s,a] + self.delta*self.v[self.sp[s,a]] for a in range(self.actions)])
        error = np.max([error, np.abs(v-self.v[s])])
      i += 1
    
    a_greedy = [np.argmax([self.r[s,a] + self.delta*self.v[self.sp[s,a]] for a in range(self.actions)]) for s in range(self.states)]
    self.p_a = np.array([[1 if a == a_greedy[s] else 0 for a in range(self.actions)] for s in range(self.states)])

    return self.p_a

  def policy_iteration(self):
    done = False
    while not done :
      self.evaluation()
      done = self.improvement()
    return self.v, self.p_a

  def evaluation(self):
    error = self.theta
    i = 0
    while error >= self.theta and i < self.steps:
      for s in range(self.states):
        v = self.v[s]
        self.v[s] = np.sum([self.p_a[s,a] * (self.r[s,a] + self.delta*self.v[self.sp[s,a]]) for a in range(self.actions)])
        error = np.max([error, np.abs(v-self.v[s])])
      i += 1

  def improvement(self) -> bool:
    stable = True
    for s in range(self.states):
      a = np.random.choice(np.arange(len(self.p_a[s])), p=self.p_a[s])
      a_greedy = np.argmax([self.r[s,a] + self.delta*self.v[self.sp[s,a]] for a in range(self.actions)])
      self.p_a[s] = np.array([1 if i==a_greedy else 0 for i in range(self.actions)])
      if a != a_greedy:
        stable = False
    return stable


In [81]:
r = np.array([
  [0, 0, 0, 0],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [-1,-1,-1,-1],
  [0, 0, 0, 0]
])
sp = np.array([
  [ 0, 0, 0, 0],
  [ 0, 1, 2, 5],
  [ 1, 2, 3, 7],
  [ 2, 3, 3, 7],
  [ 4, 0, 5, 8],
  [ 4, 1, 6, 9],
  [ 5, 2, 7,10],
  [ 6, 3, 7,11],
  [ 8, 4, 9,12],
  [ 8, 5,10,13],
  [ 9, 6,11,14],
  [10, 7,11,15],
  [12, 8,13,12],
  [12, 9,14,13],
  [13,10,15,14],
  [14,11,15,15],
])
"""
 0  1  2  3
 4  5  6  7
 8  9 10 11
12 13 14 15
"""

'\n 0  1  2  3\n 4  5  6  7\n 8  9 10 11\n12 13 14 15\n'

In [82]:
sys = DP(r, sp, method="policy", steps=10, delta=0.5, theta=1)
v, p = sys.run()
print(np.reshape(v, (4,4)))
print(p)

[[ 0.00000000e+00 -1.00000000e+00 -1.50000000e+00 -1.75000000e+00]
 [-1.00000000e+00 -1.50000000e+00 -1.75000000e+00 -1.50000057e+00]
 [-1.50000000e+00 -1.75000000e+00 -1.50000057e+00 -1.00000057e+00]
 [-1.75000000e+00 -1.50000057e+00 -1.00000057e+00 -5.67519736e-07]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


In [83]:
sys = DP(r, sp, method="value", steps=10, delta=0.5, theta=1)
p = sys.run()
print(p)

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]]
